In [2]:
import openai
import instructor

from pydantic import BaseModel, Field
from qdrant_client import QdrantClient


In [7]:
qdrant_client = QdrantClient(url="http://localhost:6333")


In [10]:
class RAGUsedContext(BaseModel):
    id: str = Field(description="ID of the item used to answer the question.")
    description: str = Field(description="Short description of the item used to answer the question.")
    
class RAGGenerationResponse(BaseModel):
    answer: str = Field(description="Answer to the user's question")
    references: list[RAGUsedContext] = Field(description="List of references to the context used to answer the question")


In [ ]:
def get_embedding(text, model="text-embedding-3-small"):
    response = openai.embeddings.create(
        input=text,
        model=model,
    )

    return response.data[0].embedding


def retrieve_data(query, qdrant_client, k=5):

    query_embedding = get_embedding(query)

    results = qdrant_client.query_points(
        collection_name="Amazon-items-collection-00",
        query=query_embedding,
        limit=k,
    )

    retrieved_context_ids = []
    retrieved_context = []
    retrieved_context_ratings = []
    similarity_scores = []

    for result in results.points:
        retrieved_context_ids.append(result.payload["parent_asin"])
        retrieved_context.append(result.payload["description"])
        retrieved_context_ratings.append(result.payload["average_rating"])
        similarity_scores.append(result.score)

    return {
        "retrieved_context_ids": retrieved_context_ids,
        "retrieved_context": retrieved_context,
        "retrieved_context_ratings": retrieved_context_ratings,
        "similarity_scores": similarity_scores,
    }


def process_context(context):

    formatted_context = ""

    for id, chunk, rating in zip(context["retrieved_context_ids"], context["retrieved_context"], context["retrieved_context_ratings"]):
        formatted_context += f"- ID: {id}, rating: {rating}, description: {chunk}\n"

    return formatted_context


def build_prompt(preprocessed_context, question):

    prompt = f"""
You are a shopping assistant that can answer questions about the products in stock.

You will be given a question and a list of context.

Instructtions:
- You need to answer the question based on the provided context only.
- Never use word context and refer to it as the available products.
- As an output you need to provide:

* The answer to the question based on the provided context.
* The list of the IDs of the chunks that were used to answer the question. Only return the ones that are used in the answer.
* Short description (1-2 sentences) of the item based on the description provided in the context.

- The short description should have the name of the item.
- The answer to the question should contain all of the relevant products in the context, returned with a bulleted list of their detailed specifications

Context:
{preprocessed_context}

Question:
{question}
"""

    return prompt

def generate_answer(prompt):

    client = instructor.from_openai(openai.OpenAI())

    response, raw_response = client.chat.completions.create_with_completion(
        model="gpt-4.1-mini",
        messages=[{"role": "system", "content": prompt}],
        temperature=0.5,
        response_model=RAGGenerationResponse
    )

    return response


def rag_pipeline(question, qdrant_client, top_k=5):

    retrieved_context = retrieve_data(question, qdrant_client, top_k)
    preprocessed_context = process_context(retrieved_context)
    prompt = build_prompt(preprocessed_context, question)
    answer = generate_answer(prompt)

    final_result = {
        "answer": answer.answer,
        "reference": answer.references,
        "question": question,
        "retrieved_context_ids": retrieved_context["retrieved_context_ids"],
        "retrieved_context_ratings": retrieved_context["retrieved_context_ratings"],
        "retrieved_context": retrieved_context["retrieved_context"],
        "similarity_scores": retrieved_context["similarity_scores"]
    }

    return final_result

In [18]:
response = rag_pipeline("Can I get women's ear rings?", qdrant_client, top_k=10)

In [15]:
response

{'answer': "Yes, you can get women's earrings. There are several options available including:\n\n- PLOMFOV 20G Stainless Steel Stud Earrings multipack with various styles (hearts, stars, flowers, etc.) suitable for multiple piercings and hypoallergenic. (ID: B09X7BVK19)\n- 925 Sterling Silver Stud Earrings Set with square and round cubic zirconia, hypoallergenic and suitable for daily wear. (ID: B0BPJ83VK1)\n- Dazzlers Sterling Silver Birthstone Stud Earrings with various simulated birthstones, hypoallergenic and comfortable. (ID: B0BY879DLN)\n\nThese options provide a range of styles and materials suitable for women.",
 'reference': [RAGUsedContext(id='B09X7BVK19', description='PLOMFOV 20G Stainless Steel Stud Earrings for Women Multipack Cartilage Earring for Men Ball Small CZ Heart Flower Star Flat Back Earrings Helix Piercing Jewelry Earrings Piercing Jewelry'),
  RAGUsedContext(id='B0BPJ83VK1', description='925 Sterling Silver Stud Earrings Set for Women and Men, Square Round Cubi

In [19]:
print(response["answer"])

Yes, there are several women's earrings available:

1. PLOMFOV 20G Stainless Steel Stud Earrings for Women Multipack:
- Includes 14 pairs of different style stainless steel stud earrings, 3 pairs cartilage hoop earrings, 2 twisted ball and star earrings, and 2 CZ inlaid cartilage earrings.
- Made of 316L stainless steel with color retention plating, hypoallergenic, no discoloration or reaction.
- Sizes: 20G (0.8mm) pin thickness, post length 6mm, flatback size 4mm, ball earring 4mm, CZ earring 4mm, heart earring 4mm, star earrings 4.5mm, butterfly earring 7x10mm, cartilage hoop inner diameter 8mm.
- Suitable for multiple piercings, comfortable as sleep earrings, and comes with an exquisite box.

2. 925 Sterling Silver Stud Earrings Set for Women and Men:
- Set of 5 pairs of round cubic zirconia stud earrings sized 3mm to 7mm.
- Made of 925 sterling silver with 18K yellow gold plating, hypoallergenic, nickel-free, and lead-free.
- Lightweight, comfortable, classic and elegant design sui